In [4]:
import boto3
from botocore.exceptions import ClientError

In [5]:
region = "us-west-1"
bucket_name = "chatbot-ld5ehom"

s3 = boto3.client("s3", region_name=region)

def create_bucket(bucket_name, region):
    try:
        location = {"LocationConstraint": region}
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration=location
        )
        print("✅ Bucket created:", bucket_name)
    except ClientError as e:
        print("❌ Bucket creation error:", e)

create_bucket("rag-example-2", region)

❌ Bucket creation error: An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [6]:
bucket_name = "chatbot-ld5ehom"
def check_bucket_connection(bucket_name):
    try: 
        response = s3.list_objects_v2(Bucket=bucket_name)
        print("Success")
        contents = response.get("Contents", [])

        if contents:
            for obj in contents:
                print(obj["Key"])
        else: 
            print("Bucket is Empty")
    except ClientError as e:
        print(e)
check_bucket_connection(bucket_name)

Success
Bucket is Empty


In [7]:
from notion_client import Client
import re

In [8]:
notion = Client(auth="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")

search_result = notion.search(
    filter={"property":"object","value":"page"},
    sort={"direction":"descending", "timestamp":"last_edited_time"}
)

result = search_result["results"][1]
title = "None"

props = result.get("properties", {})

if "title" in props:
    title_list = props["title"].get("title",[])
    if title_list:
        title = title_list[0].get("plain_text", "None")

print(result["id"])
print(title)


23695a9a-7e1b-8085-a712-c260a41eacd0
HR(인사)


In [9]:
search_result["results"][1]

{'object': 'page',
 'id': '23695a9a-7e1b-8085-a712-c260a41eacd0',
 'created_time': '2025-07-20T03:45:00.000Z',
 'last_edited_time': '2025-07-21T03:23:00.000Z',
 'created_by': {'object': 'user',
  'id': '199d872b-594c-812b-ac8b-00024f879c96'},
 'last_edited_by': {'object': 'user',
  'id': '199d872b-594c-812b-ac8b-00024f879c96'},
 'cover': None,
 'icon': None,
 'parent': {'type': 'workspace', 'workspace': True},
 'archived': False,
 'in_trash': False,
 'properties': {'title': {'id': 'title',
   'type': 'title',
   'title': [{'type': 'text',
     'text': {'content': 'HR(인사)', 'link': None},
     'annotations': {'bold': False,
      'italic': False,
      'strikethrough': False,
      'underline': False,
      'code': False,
      'color': 'default'},
     'plain_text': 'HR(인사)',
     'href': None}]}},
 'url': 'https://www.notion.so/HR-23695a9a7e1b8085a712c260a41eacd0',
 'public_url': None}

In [10]:
def extract_sort_key(page):
    title_obj = page["properties"]["title"]["title"]
    if not title_obj:
        return float("inf")
    title_text = title_obj[0]["plain_text"]
    match = re.match(r"(\d+)", title_text)
    return int(match.group(1)) if match else float("inf")

sorted_results = sorted(search_result["results"], key=extract_sort_key)

for i, result in enumerate(sorted_results, start=1):
    title="None"
    props = result.get("properties", {})
    if "title" in props: 
        title_list = props["title"].get("title", [])
        if title_list: 
            title = title_list[0].get("plain_text", "None")

print(result["id"])
print(title)


23695a9a-7e1b-804e-bf15-cd78254cfa5c
Setup


In [12]:
import os
import json

def sanitize_filename(name):
    return name.replace("/","_").replace("\\","_").replace(":","_").replace(" ","_")

def extract_leading_number(title):
    match = re.match(r"(\d+)", title)
    return int(match.group(1) if match else float("inf"))

def get_page_title(page_obj):
    props = page_obj.get("properties", {})
    title_obj = props.get("title", {}).get("title", [])
    return title_obj[0]["plain_text"] if title_obj else "None"

def block_to_json(block, notion, list_item_number=None):
    block_type = block["type"]
    result = {"type": block_type, "context": ""}

    def get_text(rich_text):
        return "".join([t["text"]["content"] for t in rich_text])
    
    if block_type == "paragraph":
        result["content"] = get_text(block["paragraph"]["rich_text"])

    elif block_type == "numbered_list_item":
        content = get_text(block["numbered_list_item"]["rich_text"])
        result["content"] = f"{list_item_number}. {content}" if list_item_number else content

    elif block_type == "bulleted_list_item":
        result["content"] = get_text(block["bulleted_list_item"]["rich_text"])

    elif block_type == "heading_1":
        result["content"] = get_text(block["heading_1"]["rich_text"])

    elif block_type == "code":
        result["content"] = get_text(block["code"]["rich_text"])

    elif block_type == "divider":
        result["content"] = "---"

    elif block_type == "table":
        table_rows = notion.blocks.children.list(block["id"])
        table_data = []

        for row in table_rows["results"]:
            if row["type"] != "table_row":
                continue
            row_cells = row["table_row"]["cells"]
            row_data = ["".join([text["text"]["content"] for text in cell]) for cell in row_cells]
            table_data.append(row_data)

        result["content"] = table_data
        result["type"] = "table"

    return result

def process_blocks_to_json(page_id, title, blocks, notion):
    json_data = {
        "id": page_id,
        "title": title,
        "page_type": "child",
        "chunks": []
    }

    list_item_number = 1
    for block in blocks["results"]:
        block_type = block["type"]
        if block_type == "numbered_list_item":
            json_block = block_to_json(block, notion, list_item_number)
            list_item_number += 1
        else:
            json_block = block_to_json(block, notion)
        json_data["chunks"].append(json_block)

    return json_data

def save_json_data(json_data, filename, folder="Notion_Export"):
    os.makedirs(folder, exist_ok=True)
    filepath = os.path.join(folder, filename)

    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(json_data, f, indent=2, ensure_ascii=False)

    print("✅ Save complete")

def export_page_to_json(page_obj, notion):
    page_id = page_obj["id"]
    title = get_page_title(page_obj)
    filename = sanitize_filename(title) + ".json"

    parent_type = page_obj.get("parent", {}).get("type")

    if parent_type == "workspace":
        children = notion.search(
            filter={"property": "object", "value": "page"},
            sort={"direction": "ascending", "timestamp": "last_edited_time"}
        )["results"]

        sub_pages = []
        for child in children:
            child_parent = child.get("parent", {})
            if child_parent.get("type") == "page_id" and child_parent.get("page_id") == page_id:
                sub_title = get_page_title(child)
                sub_pages.append(sub_title)

        sub_pages.sort(key=extract_leading_number)

        json_data = {
            "id": page_id,
            "title": title,
            "page_type": "workspace",
            "children_titles": sub_pages
        }

    else:
        blocks = notion.blocks.children.list(page_id)
        json_data = process_blocks_to_json(page_id, title, blocks, notion)

    save_json_data(json_data, filename)

for result in search_result["results"]:
    export_page_to_json(result, notion)


✅ Save complete
✅ Save complete
✅ Save complete
✅ Save complete
